<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung

In [59]:
stock_id = 'PLUG'

In [60]:
start = dt.datetime(2019,1,1)
end = dt.datetime.now()
stock = web.DataReader(stock_id, 'yahoo', start, end)

In [61]:
df = stock
df.dropna(inplace=True)

In [62]:
indicator = pd.DataFrame(df, index=df.index)

In [63]:
ema_short = 12
ema_long = 26
bollinger_avg = 20
signalline_span = 9

indicator['short_avg_ma'] = df['Close'].rolling(window=bollinger_avg).mean()
indicator['short_avg_std'] = df['Close'].rolling(window=bollinger_avg).std()

indicator['bollinger'] = df['Close'].ewm(span=bollinger_avg, adjust=False).mean()
indicator['bollinger_lower_band'] = indicator['short_avg_ma'] - (indicator['short_avg_std'] *2)
indicator['bollinger_upper_band'] = indicator['short_avg_ma'] + (indicator['short_avg_std'] *2)

indicator['ema_short'] = df['Close'].ewm(span=12, adjust=False).mean()
indicator['ema_long'] = df['Close'].ewm(span=26, adjust=False).mean()
indicator['macd'] = indicator['ema_short'] - indicator['ema_long']
indicator['signalline'] = indicator['macd'].ewm(span=signalline_span, adjust=False).mean()

In [64]:
def calculate_dead_cat_bounce():
  set_freq = '2D'

  pct_change_close = df['Close'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((indicator.signalline > indicator.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            (df.Open < indicator.bollinger_lower_band),
                            True, False)

  indicator['dead_cat_bounce'] = dead_cat_bounce

  dead_cat_bounce = indicator[(indicator['dead_cat_bounce'] == True)]

  if (dt.datetime.today() in dead_cat_bounce.index):
    send_push('Watch out, you might run into a Dead Cat Bounce!')

In [65]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()

In [66]:
#calculate_dead_cat_bounce()
bot_warn_me('Watch Out! Check Code')

{'ok': True,
 'result': {'chat': {'first_name': 'Adrian',
   'id': 908941210,
   'last_name': 'Sarwari',
   'type': 'private'},
  'date': 1616001143,
  'from': {'first_name': 'warn_me_bot',
   'id': 1790740607,
   'is_bot': True,
   'username': 'warn_me_bot'},
  'message_id': 4,
  'text': 'Test'}}